In [ ]:
d = '/media/shreyas/DATA/ML_DATA/wheelai/gtav/train/'
#d = '/media/shreyas/DATA/ML_DATA/wheelai/gtav/sample/'

x_trn_path = d + 'x_trn.bc'
y_trn_path = d + 'y_trn.bc'

s_crp_path = d + 's_crp.bc'
x_crp_path = '/home/shreyas/Downloads/data/x_crp.bc'
v_crp_path = '/home/shreyas/Downloads/data/v_crp.bc'

x_val_path = '/media/shreyas/DATA/ML_DATA/wheelai/gtav/valid/x_val.bc'
y_val_path = '/media/shreyas/DATA/ML_DATA/wheelai/gtav/valid/y_val.bc'
results_path = '/media/shreyas/DATA/ML_DATA/wheelai/gtav/results/'

In [ ]:
from __future__ import division, print_function

import os, json
from glob import glob
import numpy as np
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, load_model,  Model
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import Adam, SGD
from keras.preprocessing import image
from keras.applications import VGG16
from keras.layers import Input, Lambda, Cropping2D, Activation, ELU
from keras.layers.merge import add, concatenate

%matplotlib inline

In [ ]:
import bcolz

In [ ]:
x_trn = bcolz.open(x_trn_path)
y_trn = bcolz.open(y_trn_path)
x_val = bcolz.open(x_val_path)
y_val = bcolz.open(y_val_path)

In [ ]:
print(x_trn.shape, y_trn.shape, x_val.shape, y_val.shape)

### Generate batches with [Keras ImageDataGenerator](https://keras.io/preprocessing/image/)

In [ ]:
batch_size = 128
input_shape = (160, 320, 3)

In [ ]:
from sklearn.utils import shuffle

def get_train_batch(offset, batch_size):
    end = offset + batch_size
    return x_trn[offset:end], y_trn[offset:end]
            
def get_valid_batch(offset, batch_size):
    end = offset + batch_size
    return x_val[offset:end], y_val[offset:end]
        
def generate_batch(num_samples, batch_size=32, train=True):
    """generator return train data and valid data in batches"""
    # Loop forever so the generator never terminates 
    while 1:
        #x_trn, y_trn = shuffle(y_trn, y_trn)
        for offset in range(0, num_samples, batch_size):
            if train:
                yield (get_train_batch(offset, batch_size))
            else:
                yield (get_valid_batch(offset, batch_size))
            

#### More about Data Augmentation with Keras [here](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html) and [here.](http://machinelearningmastery.com/image-augmentation-deep-learning-keras/)

### More of Nvidia's [End to End Learning Model](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/)

In [ ]:
def nvidia_model():
    '''Implements Nvidia's end to end learning model'''
    # Croping the image to retain only road part of image
    x = Input(shape=input_shape)
    # layer to learn color space
    color_s = Conv2D(3, (1,1), padding='same')(x)
    # normalise to mean 0
    norm = Lambda(lambda x: (x / 255.0) - 0.5)(color_s)
    
    conv_b1 = Conv2D(24, (3,3), padding='same', activation='relu')(norm)
    batch_1 = BatchNormalization()(conv_b1)
    max_pl1 = MaxPooling2D()(batch_1)
    
    conv_b2 = Conv2D(36, (3,3), padding='same', activation='relu')(max_pl1)
    batch_2 = BatchNormalization()(conv_b2)
    max_pl2 = MaxPooling2D()(batch_2)
    
    conv_b3 = Conv2D(48, (3,3), padding='same', activation='relu')(max_pl2)
    batch_3 = BatchNormalization()(conv_b3)
    max_pl3 = MaxPooling2D()(batch_3)
    
    conv_b4 = Conv2D(64, (3,3), padding='same', activation='relu')(max_pl3)
    batch_4 = BatchNormalization()(conv_b4)
    max_pl4 = MaxPooling2D()(batch_4)
    dropout = Dropout(0.1)(max_pl4)
    
    conv_b5 = Conv2D(64, (3,3), padding='same', activation='relu')(dropout)
    batch_5 = BatchNormalization()(conv_b5)
    max_pl5 = MaxPooling2D((1,4))(batch_5)
    dropout = Dropout(0.1)(max_pl5)
    
    flatten = Flatten()(dropout)
    fc1 = Dense(100, activation='relu')(flatten)
    batch_fc1 = BatchNormalization()(fc1)
    drop_fc1 = Dropout(0.2)(batch_fc1)
    
    fc2 = Dense(50, activation='relu')(drop_fc1)
    batch_fc2 = BatchNormalization()(fc2)
    drop_fc2 = Dropout(0.3)(batch_fc2)
    
    fc3 = Dense(10, activation='relu')(drop_fc2)
    batch_fc3 = BatchNormalization()(fc3)
    drop_fc3 = Dropout(0.5)(batch_fc3)
    
    output = Dense(3)(drop_fc3)
    
    model = Model(inputs=x, outputs=output)
    return model

In [ ]:
def nvidia_conv():
    '''Implement a model similar to Nvidia end to end but with only conv layers'''
    model = Sequential()
    model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=input_shape))
    model.add(Conv2D(3, (1,1), padding='same')) 
    model.add(Conv2D(24, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D())
    model.add(Conv2D(36, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D())   
    model.add(Conv2D(48, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D())
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((1,4)))
    model.add(Conv2D(3, (3,3), padding='same'))
    model.add(GlobalAveragePooling2D())
    
    return model

In [ ]:
def commaai():
    #ch, row, col = 3, 160, 320  # camera format

    model = Sequential()
    model.add(Lambda(lambda x: x/127.5 - 1.,input_shape=input_shape))
    model.add(Conv2D(16, (8, 8), strides=(4, 4), padding="same"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D())
    model.add(Conv2D(32, (5, 5), strides=(2, 2), padding="same"))
    model.add(Dropout(.1))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D())
    model.add(Conv2D(64, (5, 5), strides=(2, 2), padding="same"))
    model.add(Dropout(.2))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D())
    model.add(Flatten())
    model.add(Dropout(.3))
    model.add(Dense(512))
    model.add(Dropout(.5))
    model.add(ELU())
    model.add(Dense(3))

    model.compile(optimizer="adam", loss="mse")

    return model


## 1 Nvidia Model

In [ ]:
model = nvidia_model()
model.summary()

In [ ]:
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=adam,loss='mse')

In [ ]:
# serialize model to JSON
model_path = results_path+'nvidia.json'
model_json = model.to_json()
with open(model_path, "w") as json_file:
    json_file.write(model_json)

print("Saved model to disk")

In [ ]:
trn_gen = generate_batch(x_trn.shape[0], batch_size)
val_gen = generate_batch(x_val.shape[0], batch_size, train=False)

In [ ]:
model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'nvidia1.h5')

In [ ]:
#filepath1 = results_path+'nvidia.h5'
#checkpoint1 = ModelCheckpoint(filepath1, monitor='val_loss', verbose=1, save_best_only=False,\
#                             save_weights_only=True, mode='min', period=1)
#callbacks1=[checkpoint1]

In [ ]:
model.optimizer.lr = 1e-5
model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'nvidia2.h5')

In [ ]:
model.optimizer.lr = 1e-3
model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'nvidia3.h5')

In [ ]:
#filepath2 = results_path+'nvidia_aug.h5'
#checkpoint2 = ModelCheckpoint(filepath2, monitor='val_loss', verbose=1, save_best_only=False,\
#                             save_weights_only=True, mode='min', period=1)
#callbacks2=[checkpoint2]

In [ ]:
model.optimizer.lr = 1e-5
model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'nvidia4.h5')

In [ ]:
model.optimizer.lr = 1e-4
model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'nvidia5.h5')

In [ ]:
model.optimizer.lr = 1e-5
model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'nvidia6.h5')

## 2 Nvidia All CONV model

In [ ]:
model = nvidia_conv()
model.summary()

In [ ]:
model.compile(optimizer="adam", loss="mse")

In [ ]:
# serialize model to JSON
model_path = results_path+'nvidia_conv.json'
model_json = model.to_json()
with open(model_path, "w") as json_file:
    json_file.write(model_json)

print("Saved model to disk")

In [ ]:
trn_gen = generate_batch(x_trn.shape[0], batch_size)
val_gen = generate_batch(x_val.shape[0], batch_size, train=False)

In [ ]:
model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'nvidia_conv1.h5')

In [ ]:
model.optimizer.lr = 1e-5
model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'nvidia_conv2.h5')

In [ ]:
model.optimizer.lr = 1e-3
model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'nvidia_conv3.h5')

In [ ]:
#filepath2 = results_path+'nvidia_aug.h5'
#checkpoint2 = ModelCheckpoint(filepath2, monitor='val_loss', verbose=1, save_best_only=False,\
#                             save_weights_only=True, mode='min', period=1)
#callbacks2=[checkpoint2]

In [ ]:
model.optimizer.lr = 1e-5
model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'nvidia_conv4.h5')

In [ ]:
model.optimizer.lr = 1e-4
model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'nvidia_conv5.h5')

In [ ]:
model.optimizer.lr = 1e-5
model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'nvidia_conv6.h5')

## 3 CommaAI Model

In [ ]:
model = commaai()
model.summary()

In [ ]:
# serialize model to JSON
model_path = results_path+'commaai.json'
model_json = model.to_json()
with open(model_path, "w") as json_file:
    json_file.write(model_json)

print("Saved model to disk")

In [ ]:
trn_gen = generate_batch(x_trn.shape[0], batch_size)
val_gen = generate_batch(x_val.shape[0], batch_size, train=False)

In [ ]:
model.load_weights(results_path+'commaai6.h5')

In [ ]:

model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'commaai1.h5')

In [ ]:
model.optimizer.lr = 1e-5
model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'commaai2.h5')

In [ ]:
model.optimizer.lr = 1e-3
model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'commaai3.h5')

In [ ]:
#filepath2 = results_path+'nvidia_aug.h5'
#checkpoint2 = ModelCheckpoint(filepath2, monitor='val_loss', verbose=1, save_best_only=False,\
#                             save_weights_only=True, mode='min', period=1)
#callbacks2=[checkpoint2]

In [ ]:
model.optimizer.lr = 1e-4
model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'commaai4.h5')

In [ ]:
model.optimizer.lr = 1e-3
model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'commaai5.h5')

In [ ]:
model.optimizer.lr = 1e-5
model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'commaai6.h5')

In [ ]:
model.optimizer.lr = 1e-5
model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'commaai7.h5')

In [ ]:
model.optimizer.lr = 1e-5
model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'commaai8.h5')